# Doc2Vec을 기반한 뉴스 연관 추천

## 1. 뉴스 데이터 로드

In [9]:
from doc2vec.util.Logger import Logger
from doc2vec.util.conf_parser import Parser
from doc2vec.elastic.analyzer.nori_analyzer import NoriAnalyzer
from doc2vec.preprocessing.preprocessing import *
from doc2vec.util.common import load_pickle, save_pickle
from doc2vec.preprocessing.preprocessing import generate_tagged_document_by_pandas, load_pdf, is_contain_str
import pandas as pd 
from doc2vec.model.doc2vec import build_model

In [ ]:
logger = Logger(file_name=__name__).logger
p = Parser()

nori = NoriAnalyzer(**p.elastic_conf)

In [4]:
ori_df = load_pdf("news.csv", sep="^")

# 제거할 거 (news_nm기준으로 제거, 65,144 -> 64,712로 432개 article이 제거됨 )
# - 클로징, BGM, 썰전 라이브 다시보기(-> 이건 text 내용이 없음)
# 사용자 history 기반 news 수 :
#          news_id  ...                                       news_content
# 65139  NB12045093  ...  <div id='div_NV10478125' ...
# 65140  NB12045094  ...  <div id='div_NV10478127' ...
ori_df['is_in_str'] = ori_df['news_nm'].apply(is_contain_str)
df = ori_df[~ori_df['is_in_str']==True].copy()
df.reset_index(drop=True, inplace=True)

df.tail()



file_path : /Users/jmac/project/jtbc_news_data/analyzer/data/rec/news.csv
file_name : news.csv
DATA INFO
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65144 entries, 0 to 65143
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype                                
---  ------        --------------  -----                                
 0   news_id       65144 non-null  object                               
 1   news_nm       65144 non-null  object                               
 2   section       65144 non-null  object                               
 3   service_date  65144 non-null  datetime64[ns, pytz.FixedOffset(540)]
 4   news_content  65144 non-null  object                               
dtypes: datetime64[ns, pytz.FixedOffset(540)](1), object(4)
memory usage: 2.5+ MB

----------------------------------------------------------------------------------------------------
DATA IS_NULL
+--------------+-----+
|              |   0 |
|--------------+-----|
| news_id 

,news_id,news_nm,section,service_date,news_content,is_in_str
64707,NB12045093,설 전 양자토론 끝내 무산됐다…남은 건 '네 탓 공방',정치,2022-01-31 19:44:00+09:00,<div id='div_NV10478125' class='jtbc_vod'></di...,False
64708,NB12045094,"[인터뷰] 박주민 ""자료 있어야 토론, 후보가 준비 안 된 것""",정치,2022-01-31 21:19:00+09:00,<div id='div_NV10478127' class='jtbc_vod'></di...,False
64709,NB12045095,"[인터뷰] 성일종 ""범죄 혐의 자료, 증거 제시 위해 필요""",정치,2022-01-31 21:24:00+09:00,<div id='div_NV10478129' class='jtbc_vod'></di...,False
64710,NB12045096,1월 31일 (월) 뉴스룸 다시보기,정치,2022-02-05 16:25:00+09:00,<div id='div_NV10478712' class='jtbc_vod'></di...,False
64711,NB12045105,가정집에서 전 부치다 부탄가스 폭발…일가족 7명 부상,사회,2022-01-31 20:25:00+09:00,<div id='div_NV10478151' class='jtbc_vod'></di...,False


## 2. elastic 기반 형태소 분석 수행 (with. nori analyzer)

위에서 로드한 news.csv 파일에서 news_nm과 news_content를 기준으로 사전에 구축 및 nori 플러그인이 설치된 elastic에 analyzer를 수행하여
형태소 분석된 결과를 parsing_data_df(pandas) 형태로 저장
- 참고로 elastic analyze에서는 list 형태로 데이터 분석에 제한이 있어서 우선 step=10으로 10개의 documents 단위로 형태소 분석을 수행함

In [ ]:
save_flag = True

step = 10

analyzed_df = pd.DataFrame(columns=['news_id', 'tagged_doc'])

news_contents_parsing = {}
nori.update_index_setting()
for idx in range(0, df.shape[0], step):
    if idx + step > df.shape[0]:
        parsing_df = df.iloc[idx: df.shape[0]][['news_id', 'news_content', 'news_nm']]
    else:
        parsing_df = df.iloc[idx: idx + step][['news_id', 'news_content', 'news_nm']]

    tmp_parsing = nori.get_parsing_news_contents_from_csv(parsing_df)
    news_contents_parsing.update(tmp_parsing)
    result_df = pd.DataFrame({'news_id': list(tmp_parsing.keys()), 'tagged_doc': list(tmp_parsing.values())})
    analyzed_df = pd.concat([analyzed_df, result_df], axis=0)

if save_flag:
    save_pickle(os.path.join(rootpath.detect(), *["data", "rec", "parsing_data.pickle"]), news_contents_parsing)
    save_pickle(os.path.join(rootpath.detect(), *["data", "rec", "parsing_data_df.pickle"]),
                analyzed_df)

## 3. doc2vec 모델 생성을 위한 데이터 변환 및 분할

elastic기반으로 형태소 분석된 데이터를 TaggedDocument로 변환하고
검증을 위해 훈련데이터와 검증 데이터를 나뉨

TaggedDocument는 words와 tags로 정의 됨
- TaggedDocument(['홍남기', '부총리', '기획', '재정부', '장관', ..., '통보'], ['2022030610442141299'])

In [10]:
from sklearn.model_selection import train_test_split

parsing_data_df_path = os.path.join(rootpath.detect(), *["data", "rec", "parsing_data_df.pickle"])
parsing_data_df = load_pickle(parsing_data_df_path)

x_train, x_test, y_train, y_test = train_test_split(parsing_data_df[['tagged_doc']],
                                                    parsing_data_df['news_id'],
                                                    test_size=0.3,
                                                    shuffle=True,
                                                    random_state=42)
x_train = x_train.reset_index(drop=True)
x_test = x_test.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)
print("x_train.shape= ", x_train.shape, ", y_train.shape= ", y_train.shape)
print("x_test.shape= ", x_test.shape, ", y_test.shape= ", y_test.shape)

train_tagged_doc = generate_tagged_document_by_pandas(x_train, y_train)
test_tagged_doc = generate_tagged_document_by_pandas(x_test, y_test)

save_pickle(os.path.join(rootpath.detect(), *["data", "rec", "train_tagged_doc.pickle"]), train_tagged_doc)
save_pickle(os.path.join(rootpath.detect(), *["data", "rec", "test_tagged_doc.pickle"]), test_tagged_doc)

x_train.shape=  (45298, 1) , y_train.shape=  (45298,)
x_test.shape=  (19414, 1) , y_test.shape=  (19414,)


## 4. 모델 생성 및 저장

doc2vec 모델을 위해서 gensim을 사용하였으며, 모델에 대한 정의는 doc2vec/model/doc2vec.py에 정의 되어 있음

일반적으로 dm=1 즉, PV-DM 학습 방법이 성능이 더 좋다고 하여 해당 방법을 채택

또한 vector_size가 크면 좋지만, 그에 따른 학습 속도가 길어지기 때문에 vector_size와 windows 사이즈에 대한 성능 측정을 참고하여 vector_size=1000, window=3을 선택


참고: 내부에서 정의된 모델 정보 (병렬처리 하게끔 설정함)


    model = gensim.models.doc2vec.Doc2Vec(vector_size=1000, min_count=2, epochs=100,
                                            alpha=0.025, min_alpha=0.00025, workers=cores,
                                            window=3, dm=1, seed=9999)
                                


In [ ]:
tagged_doc = load_pickle(os.path.join(rootpath.detect(), *["data", "rec", "train_tagged_doc.pickle"]))

# 모델 불러오기
d2v_model = build_model()

# 사전 구축
d2v_model.build_vocab(tagged_doc)

# 모델 훈련
d2v_model.train(tagged_doc, total_examples=d2v_model.corpus_count, epochs=d2v_model.epochs)

d2v_model.save(os.path.join(rootpath.detect(), *['model', 'model.doc2vec']))

모델 테스트는 아래와 같이 가볍게 할 수 있음:

- 아래는 검증 데이터를 기반으로 훈련된 모델에서 문서에 대한 vector가 주어졌을때 vector를 뽑아줌

In [12]:
test_tagged_doc = load_pickle(os.path.join(rootpath.detect(), *["data", "rec", "test_tagged_doc.pickle"]))

d2v_model = gensim.models.doc2vec.Doc2Vec.load(os.path.join(rootpath.detect(), *['model', 'model.doc2vec']))

print(test_tagged_doc[0])
vector = d2v_model.infer_vector(test_tagged_doc[0].words)
print("vector size = ", len(vector))
print("Top 10 values in Doc2Vec inferred vecotr: ")
print(vector[:10])

2022-03-14 15:57:20,508 [INFO] - utils.py:load - line:481 - loading Doc2Vec object from /Users/jmac/project/jtbc_news_data/analyzer/model/model.doc2vec
2022-03-14 15:57:20,572 [INFO] - utils.py:_load_specials - line:515 - loading dv recursively from /Users/jmac/project/jtbc_news_data/analyzer/model/model.doc2vec.dv.* with mmap=None
2022-03-14 15:57:20,573 [INFO] - utils.py:_load_specials - line:515 - loading wv recursively from /Users/jmac/project/jtbc_news_data/analyzer/model/model.doc2vec.wv.* with mmap=None
2022-03-14 15:57:20,574 [INFO] - utils.py:_load_specials - line:520 - loading vectors from /Users/jmac/project/jtbc_news_data/analyzer/model/model.doc2vec.wv.vectors.npy with mmap=None
2022-03-14 15:57:20,812 [INFO] - utils.py:_load_specials - line:520 - loading syn1neg from /Users/jmac/project/jtbc_news_data/analyzer/model/model.doc2vec.syn1neg.npy with mmap=None
2022-03-14 15:57:21,088 [INFO] - utils.py:_load_specials - line:553 - setting ignored attribute cum_table to None
202

TaggedDocument(['김영희', '성탄절', '이재명', '영상', '공개', '민주당', '선대위', '홍보', '소통', '본부', '장', '영입', '김영희', 'MBC', '콘텐츠', '총괄', '부사장', '오른쪽', '사진', '연합뉴스', 'MBC', '콘텐츠', '총괄', '부사장', '김영희', '민주당', '선거', '대책', '위원회', '홍보', '소통', '본부', '장', '25', '일', '모두', '이재명', '대선', '후보', '기획', '영상', '공개', '김', '본부', '장', '어제', '17', '일', 'CBS', '라디오', '출연', '후보', '설득', '선거', '판', '대선', '중요', '생각', '힘든', '시도', '말', '내용', '인지', '힌트', '질문', '그', '말씀', '누구', '카피', '수준', '라면서', '후보', '산타나', '루돌프', '복장', '냐는', '질문', '그럴', '말', '김', '본부', '장', '후보', '유능', '경제', '대통령', '생각', '후보', '인간', '인', '모습', '있', '유능', '경제', '대통령', '친근', '부드러운', '이미지', '승리', '이', '생각', '선대위', '합류', '날', '일요일', '집', '앞', '송영길', '민주당', '대표', '밤', '8', '시', '1', '시간', '이건', '예의', '아닌', '같', '집', '맥주', '배경', '설명', '그', '이준석', '국민의힘', '대표', '집', '앞', '2', '시간', '그쪽', '질문', '그쪽', '마음', '말'], ['NB12039441'])
vector size =  1000
Top 10 values in Doc2Vec inferred vecotr: 
[-0.23173319  0.52654815  0.16003528  0.3549234  -0.39076102  0.58109957
 -0.1372

In [14]:
# 모델에 훈련된 tags를 확인 하는 방법
d2v_model = gensim.models.doc2vec.Doc2Vec.load(os.path.join(rootpath.detect(), *['model', 'model.doc2vec']))
print(d2v_model.docvecs.index_to_key[:10])

2022-03-14 15:57:49,929 [INFO] - utils.py:load - line:481 - loading Doc2Vec object from /Users/jmac/project/jtbc_news_data/analyzer/model/model.doc2vec
2022-03-14 15:57:49,961 [INFO] - utils.py:_load_specials - line:515 - loading dv recursively from /Users/jmac/project/jtbc_news_data/analyzer/model/model.doc2vec.dv.* with mmap=None
2022-03-14 15:57:49,962 [INFO] - utils.py:_load_specials - line:515 - loading wv recursively from /Users/jmac/project/jtbc_news_data/analyzer/model/model.doc2vec.wv.* with mmap=None
2022-03-14 15:57:49,963 [INFO] - utils.py:_load_specials - line:520 - loading vectors from /Users/jmac/project/jtbc_news_data/analyzer/model/model.doc2vec.wv.vectors.npy with mmap=None
2022-03-14 15:57:50,054 [INFO] - utils.py:_load_specials - line:520 - loading syn1neg from /Users/jmac/project/jtbc_news_data/analyzer/model/model.doc2vec.syn1neg.npy with mmap=None
2022-03-14 15:57:50,146 [INFO] - utils.py:_load_specials - line:553 - setting ignored attribute cum_table to None
202

['NB12039792', 'NB12038499', 'NB12031313', 'NB12044649', 'NB12034031', 'NB12034896', 'NB12034798', 'NB12043956', 'NB12038338', 'NB12039392']


/var/folders/9m/hqzkkj0n7030by4xtpmq_fcw0000gn/T/ipykernel_82095/780563596.py:3: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  print(d2v_model.docvecs.index_to_key[:10])


## 5. 연관 문서 추천 (doc2vec)

훈련 데이터를 기반으로 특정 문서를 넣었을때, 해당 문서와 유사한 문서를 훈련된 모델로 찾음

참고: 자기 자신 제외는 아직 로직 상 집어 넣지 않았음.

(그래도 결과적으로 유사한 뉴스가 나옴)


In [23]:
def _check_sim_news(data, news_df, d2v_model):

    for a_data in data:
        selected_news_id = a_data.tags[0]
        selected_news_df = news_df[news_df['news_id'] == selected_news_id][['news_id', 'news_nm', 'news_content']]
        print("\n=============================================================================================================")
        print(f"[{selected_news_id}] {selected_news_df.news_nm.values[0]}")
        inferred = d2v_model.infer_vector(a_data.words)
        sims = d2v_model.dv.most_similar(inferred, topn=5)

        for sim in sims:
            news_id = sim[0]
            sim_value = sim[1]
            tmp_sim_news_df = news_df[news_df['news_id'] == news_id][
                ['news_id', 'news_nm', 'news_content']]
            if not tmp_sim_news_df.empty:
                print(f" 연관추천 [{news_id} ({sim_value})] {tmp_sim_news_df.news_nm.values[0]}")
        print("\n\n")

In [26]:
news_df = load_pdf('news.csv', sep="^")

d2v_model = gensim.models.doc2vec.Doc2Vec.load(os.path.join(rootpath.detect(), *['model', 'model.doc2vec']))
# data = load_pickle(os.path.join(rootpath.detect(), *["data", "rec","parsing_data.pickle"]))
data = load_pickle(os.path.join(rootpath.detect(), *["data", "rec", "test_tagged_doc.pickle"]))

_check_sim_news(data[101:106], news_df, d2v_model)

file_path : /Users/jmac/project/jtbc_news_data/analyzer/data/rec/news.csv
file_name : news.csv
DATA INFO
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65144 entries, 0 to 65143
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype                                
---  ------        --------------  -----                                
 0   news_id       65144 non-null  object                               
 1   news_nm       65144 non-null  object                               
 2   section       65144 non-null  object                               
 3   service_date  65144 non-null  datetime64[ns, pytz.FixedOffset(540)]
 4   news_content  65144 non-null  object                               
dtypes: datetime64[ns, pytz.FixedOffset(540)](1), object(4)
memory usage: 2.5+ MB

----------------------------------------------------------------------------------------------------
DATA IS_NULL
+--------------+-----+
|              |   0 |
|--------------+-----|
| news_id 

2022-03-14 16:36:36,483 [INFO] - utils.py:load - line:481 - loading Doc2Vec object from /Users/jmac/project/jtbc_news_data/analyzer/model/model.doc2vec
2022-03-14 16:36:36,563 [INFO] - utils.py:_load_specials - line:515 - loading dv recursively from /Users/jmac/project/jtbc_news_data/analyzer/model/model.doc2vec.dv.* with mmap=None
2022-03-14 16:36:36,565 [INFO] - utils.py:_load_specials - line:515 - loading wv recursively from /Users/jmac/project/jtbc_news_data/analyzer/model/model.doc2vec.wv.* with mmap=None
2022-03-14 16:36:36,566 [INFO] - utils.py:_load_specials - line:520 - loading vectors from /Users/jmac/project/jtbc_news_data/analyzer/model/model.doc2vec.wv.vectors.npy with mmap=None
2022-03-14 16:36:36,654 [INFO] - utils.py:_load_specials - line:520 - loading syn1neg from /Users/jmac/project/jtbc_news_data/analyzer/model/model.doc2vec.syn1neg.npy with mmap=None
2022-03-14 16:36:36,739 [INFO] - utils.py:_load_specials - line:553 - setting ignored attribute cum_table to None
202


[NB12042908] [걸어서 인터뷰ON] 이준석 "김건희 심신피폐, 국민여론 일부 호응할 수도"
 연관추천 [NB12042908 (0.9250392913818359)] [걸어서 인터뷰ON] 이준석 "김건희 심신피폐, 국민여론 일부 호응할 수도"
 연관추천 [NB12042931 (0.5079581141471863)] 오늘 '김건희 녹취' 방송…이준석 "일정부분 공감 여론 생길 것"
 연관추천 [NB12031492 (0.3708450496196747)] 11월 8일 (월) 뉴스룸 다시보기
 연관추천 [NB12044284 (0.3635936975479126)] [속보]'요양급여 불법 수급' 윤석열 장모 2심서 무죄
 연관추천 [NB12042836 (0.3451680839061737)] 1월 14일 (금) 뉴스룸 다시보기




[NB12040914] [뉴스썰기] 조원진 "야당 후보교체"…'무야홍' 어게인? 
 연관추천 [NB12040914 (0.8656050562858582)] [뉴스썰기] 조원진 "야당 후보교체"…'무야홍' 어게인? 
 연관추천 [NB12044284 (0.47114697098731995)] [속보]'요양급여 불법 수급' 윤석열 장모 2심서 무죄
 연관추천 [NB12035244 (0.41691717505455017)] [날씨] 전국 대체로 맑고 큰 일교차…동해안 건조특보
 연관추천 [NB12039793 (0.41420161724090576)] [오늘, 이 장면] 바람을 타는지, 파도를 타는지 모르는 '카이트 보딩'
 연관추천 [NB12040157 (0.41247880458831787)] 문 대통령, 박근혜 전격 사면할듯…한명숙도 포함




[NB12035070] 한때 배달원이던 이 선수, 꿈의 무대서 극적 결승골
 연관추천 [NB12035070 (0.8485115766525269)] 한때 배달원이던 이 선수, 꿈의 무대서 극적 결승골
 연관추천 [NB12041512 (0.4686205983161926)] [오늘, 이 장면] 덩크슛은 높아야? 